# Assignment 2
## Lace Grant, Section 2

In [31]:
# import the two Gurobi parts we need
import gurobipy as gp
from gurobipy import GRB
import numpy as np

### 1. What distribution pattern minimizes overall cost? What is the overall cost?

First, we input the data from the problem so that we are able to solve it.

In [16]:
# lists
factories = ["Cleveland", "Austin"]
distribution_centers = ["Pittsburgh", "Boulder", "Des Moines", "Tucson"]
customers = ["Customer1", "Customer2", "Customer3", "Customer4", "Customer5", "Customer6"]

#factory capacity
s_i = {"Cleveland": 150000, "Austin": 200000}

#dc capacity
t_j = {"Pittsburgh": 70000, "Boulder": 50000, "Des Moines": 100000, "Tucson": 40000}

#customer demand
d_k = {
    "Customer1": 50000,
    "Customer2": 10000,
    "Customer3": 40000,
    "Customer4": 35000,
    "Customer5": 60000,
    "Customer6": 20000,
}

#transportation costs
c_ij = {("Cleveland", "Pittsburgh"): 0.50, ("Cleveland", "Boulder"): 0.50, 
        ("Cleveland", "Des Moines"): 1.00, ("Cleveland", "Tucson"): 0.20,
        ("Austin", "Pittsburgh"): None, ("Austin", "Boulder"): 0.30,
        ("Austin", "Des Moines"): 0.50, ("Austin", "Tucson"): 0.20}
e_ik = {("Cleveland", "Customer1"): 1.00, ("Cleveland", "Customer2"): None,
        ("Cleveland", "Customer3"): 1.50, ("Cleveland", "Customer4"): 2.00,
        ("Cleveland", "Customer5"): None, ("Cleveland", "Customer6"): 1.00,
        ("Austin", "Customer1"): 2.00, ("Austin", "Customer2"): 1.50,
        ("Austin", "Customer3"): 0.50, ("Austin", "Customer4"): 1.50,
        ("Austin", "Customer5"): 0.50, ("Austin", "Customer6"): 1.00}
f_jk = {("Pittsburgh", "Customer1"): None, ("Pittsburgh", "Customer2"): 1.50,
        ("Pittsburgh", "Customer3"): 0.50, ("Pittsburgh", "Customer4"): 1.50,
        ("Pittsburgh", "Customer5"): None, ("Pittsburgh", "Customer6"): None,
        ("Boulder", "Customer1"): 1.00, ("Boulder", "Customer2"): 0.50,
        ("Boulder", "Customer3"): 0.50, ("Boulder", "Customer4"): 1.00,
        ("Boulder", "Customer5"): 0.50, ("Boulder", "Customer6"): None,
        ("Des Moines", "Customer1"): None, ("Des Moines", "Customer2"): 1.50,
        ("Des Moines", "Customer3"): 2.00, ("Des Moines", "Customer4"): None,
        ("Des Moines", "Customer5"): 0.50, ("Des Moines", "Customer6"): 1.50,
        ("Tucson", "Customer1"): None, ("Tucson", "Customer2"): None,
        ("Tucson", "Customer3"): 0.20, ("Tucson", "Customer4"): 1.50,
        ("Tucson", "Customer5"): 0.50, ("Tucson", "Customer6"): 1.50}

# cost factory to dc
factory_to_dc_costs = {
    ("Cleveland", "Pittsburgh"): 0.50,
    ("Cleveland", "Boulder"): 0.50,
    ("Cleveland", "Des Moines"): 1.00,
    ("Cleveland", "Tucson"): 0.20,
    ("Austin", "Pittsburgh"): None,  
    ("Austin", "Boulder"): 0.30,
    ("Austin", "Des Moines"): 0.50,
    ("Austin", "Tucson"): 0.20,
}

# cost factory to customer
factory_to_customer_costs = {
    ("Cleveland", "Customer1"): 1.00,
    ("Cleveland", "Customer2"): None,
    ("Cleveland", "Customer3"): 1.50,
    ("Cleveland", "Customer4"): 2.00,
    ("Cleveland", "Customer5"): None,
    ("Cleveland", "Customer6"): 1.00,
    ("Austin", "Customer1"): 2.00,
    ("Austin", "Customer2"): 1.50,
    ("Austin", "Customer3"): 0.50,
    ("Austin", "Customer4"): 1.50,
    ("Austin", "Customer5"): 0.50,
    ("Austin", "Customer6"): 1.00,
}

# cost dc to customer
dc_to_customer_costs = {
    ("Pittsburgh", "Customer1"): None,  # Not possible
    ("Pittsburgh", "Customer2"): 1.50,
    ("Pittsburgh", "Customer3"): 0.50,
    ("Pittsburgh", "Customer4"): 1.50,
    ("Pittsburgh", "Customer5"): None,
    ("Pittsburgh", "Customer6"): None,
    ("Boulder", "Customer1"): 1.00,
    ("Boulder", "Customer2"): 0.50,
    ("Boulder", "Customer3"): 0.50,
    ("Boulder", "Customer4"): 1.00,
    ("Boulder", "Customer5"): 0.50,
    ("Boulder", "Customer6"): None,
    ("Des Moines", "Customer1"): None,
    ("Des Moines", "Customer2"): 1.50,
    ("Des Moines", "Customer3"): 2.00,
    ("Des Moines", "Customer4"): None,
    ("Des Moines", "Customer5"): 0.50,
    ("Des Moines", "Customer6"): 1.50,
    ("Tucson", "Customer1"): None,
    ("Tucson", "Customer2"): None,
    ("Tucson", "Customer3"): 0.20,
    ("Tucson", "Customer4"): 1.50,
    ("Tucson", "Customer5"): 0.50,
    ("Tucson", "Customer6"): 1.50,
}

Then, we create our model using Gurobi.

In [17]:
#create model object
m = gp.Model('Shipping')

#specific obj function and time limit
m.ModelSense = GRB.MINIMIZE
m.setParam('TimeLimit', 600)
m.update()

Set parameter TimeLimit to value 600


In [18]:
#decision variables
x_ij = m.addVars(factories, distribution_centers, vtype=GRB.CONTINUOUS, name="x_ij")
y_ik = m.addVars(factories, customers, vtype=GRB.CONTINUOUS, name="y_ik")
z_jk = m.addVars(distribution_centers, customers, vtype=GRB.CONTINUOUS, name="z_jk")

#obj function
m.setObjective(
    gp.quicksum(c_ij[i, j] * x_ij[i, j] for i in factories for j in distribution_centers if c_ij[i, j] is not None) +
    gp.quicksum(e_ik[i, k] * y_ik[i, k] for i in factories for k in customers if e_ik[i, k] is not None) +
    gp.quicksum(f_jk[j, k] * z_jk[j, k] for j in distribution_centers for k in customers if f_jk[j, k] is not None)
)
m.update()

In [19]:
#constraints
# factory capacity
for i in factories:
    m.addConstr(
        gp.quicksum(x_ij[i, j] for j in distribution_centers if (i, j) in c_ij) +
        gp.quicksum(y_ik[i, k] for k in customers if (i, k) in e_ik) <= s_i[i],
        name=f"FactoryCapacity_{i}"
    )
# dc capacity
for j in distribution_centers:
    m.addConstr(
        gp.quicksum(x_ij[i, j] for i in factories if (i, j) in c_ij) <= t_j[j],
        name=f"DCCapacity_{j}"
    )
# dc flow
for j in distribution_centers:
    m.addConstr(
        gp.quicksum(z_jk[j, k] for k in customers if (j, k) in f_jk) ==
        gp.quicksum(x_ij[i, j] for i in factories if (i, j) in c_ij),
        name=f"FlowConservation_{j}"
    )
# customer demand
for k in customers:
    m.addConstr(
        gp.quicksum(y_ik[i, k] for i in factories if (i, k) in e_ik) +
        gp.quicksum(z_jk[j, k] for j in distribution_centers if (j, k) in f_jk) == d_k[k],
        name=f"CustomerDemand_{k}"
    )
m.update()

In [20]:
m.optimize()
if m.status == GRB.OPTIMAL:
    print("Optimal total cost:", m.objVal)
    for v in m.getVars():
        if v.X > 0:  # print only non-zero variables
            print(f"{v.VarName}: {v.X}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16 rows, 44 columns and 96 nonzeros
Model fingerprint: 0x3094d5a4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve time: 0.01s
Presolved: 16 rows, 44 columns, 96 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+03   1.450000e+05   0.000000e+00      0s
       4    3.3500000e+04   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.350000000e+04
Optimal total cost: 33500.0
x_ij[Cleveland,Tucson]: 40000.0
x_ij[Austin,Pittsburgh]: 70000.0
x_ij[Austin,Des Moines]: 35000.0
y_ik[Cleveland,Customer2]: 10

In [21]:
# check factory capacity
print("Factory Capacity Constraints:")
for i in factories:
    constr = m.getConstrByName(f"FactoryCapacity_{i}")
    lhs = sum(x_ij[i, j].X for j in distribution_centers if (i, j) in c_ij) + \
          sum(y_ik[i, k].X for k in customers if (i, k) in e_ik)
    rhs = s_i[i]
    print(f"{i} - LHS: {lhs}, RHS: {rhs}, Binding: {abs(lhs - rhs) < 1e-6}, Dual Value: {constr.Pi}")

# check distribution center
print("\nDistribution Center Capacity Constraints:")
for j in distribution_centers:
    constr = m.getConstrByName(f"DCCapacity_{j}")
    lhs = sum(x_ij[i, j].X for i in factories if (i, j) in c_ij)
    rhs = t_j[j]
    print(f"{j} - LHS: {lhs}, RHS: {rhs}, Binding: {abs(lhs - rhs) < 1e-6}, Dual Value: {constr.Pi}")

# check customer demand
print("\nCustomer Demand Constraints:")
for k in customers:
    constr = m.getConstrByName(f"CustomerDemand_{k}")
    lhs = sum(y_ik[i, k].X for i in factories if (i, k) in e_ik) + \
          sum(z_jk[j, k].X for j in distribution_centers if (j, k) in f_jk)
    rhs = d_k[k]
    print(f"{k} - LHS: {lhs}, RHS: {rhs}, Binding: {abs(lhs - rhs) < 1e-6}, Dual Value: {constr.Pi}")

# check flow conservation
print("\nFlow Conservation Constraints at Distribution Centers:")
for j in distribution_centers:
    constr = m.getConstrByName(f"FlowConservation_{j}")
    inflow = sum(x_ij[i, j].X for i in factories if (i, j) in c_ij)
    outflow = sum(z_jk[j, k].X for k in customers if (j, k) in f_jk)
    print(f"{j} - Inflow: {inflow}, Outflow: {outflow}, Binding: {abs(inflow - outflow) < 1e-6}, Dual Value: {constr.Pi}")


Factory Capacity Constraints:
Cleveland - LHS: 110000.0, RHS: 150000, Binding: False, Dual Value: 0.0
Austin - LHS: 105000.0, RHS: 200000, Binding: False, Dual Value: 0.0

Distribution Center Capacity Constraints:
Pittsburgh - LHS: 70000.0, RHS: 70000, Binding: True, Dual Value: 0.0
Boulder - LHS: 0.0, RHS: 50000, Binding: False, Dual Value: 0.0
Des Moines - LHS: 35000.0, RHS: 100000, Binding: False, Dual Value: 0.0
Tucson - LHS: 40000.0, RHS: 40000, Binding: True, Dual Value: 0.0

Customer Demand Constraints:
Customer1 - LHS: 50000.0, RHS: 50000, Binding: True, Dual Value: -0.0
Customer2 - LHS: 10000.0, RHS: 10000, Binding: True, Dual Value: -0.0
Customer3 - LHS: 40000.0, RHS: 40000, Binding: True, Dual Value: 0.4
Customer4 - LHS: 35000.0, RHS: 35000, Binding: True, Dual Value: 0.5
Customer5 - LHS: 60000.0, RHS: 60000, Binding: True, Dual Value: -0.0
Customer6 - LHS: 20000.0, RHS: 20000, Binding: True, Dual Value: -0.0

Flow Conservation Constraints at Distribution Centers:
Pittsburgh

### 3. What is the effect of increasing the factory capacities on the overall cost? Carefully and fully explain your answer and how you arrived at it.

I created a new model (n) and doubled factory capacities.

In [22]:
# updated factory capacities
new_s_i = {"Cleveland": 300000, "Austin": 400000} #doubled factory capacity for this test

#create model object
n = gp.Model('Shipping')

#specific obj function and time limit
n.ModelSense = GRB.MINIMIZE
n.setParam('TimeLimit', 600)
n.update()

#decision variables
x_ij = n.addVars(factories, distribution_centers, vtype=GRB.CONTINUOUS, name="x_ij")
y_ik = n.addVars(factories, customers, vtype=GRB.CONTINUOUS, name="y_ik")
z_jk = n.addVars(distribution_centers, customers, vtype=GRB.CONTINUOUS, name="z_jk")

#obj function
n.setObjective(
    gp.quicksum(c_ij[i, j] * x_ij[i, j] for i in factories for j in distribution_centers if c_ij[i, j] is not None) +
    gp.quicksum(e_ik[i, k] * y_ik[i, k] for i in factories for k in customers if e_ik[i, k] is not None) +
    gp.quicksum(f_jk[j, k] * z_jk[j, k] for j in distribution_centers for k in customers if f_jk[j, k] is not None)
)
n.update()

# factory capacity (UPDATED)
for i in factories:
    n.addConstr(
        gp.quicksum(x_ij[i, j] for j in distribution_centers if (i, j) in c_ij) +
        gp.quicksum(y_ik[i, k] for k in customers if (i, k) in e_ik) <= new_s_i[i],
        name=f"FactoryCapacity_{i}"
    )
# dc capacity
for j in distribution_centers:
    n.addConstr(
        gp.quicksum(x_ij[i, j] for i in factories if (i, j) in c_ij) <= t_j[j],
        name=f"DCCapacity_{j}"
    )
# dc flow
for j in distribution_centers:
    n.addConstr(
        gp.quicksum(z_jk[j, k] for k in customers if (j, k) in f_jk) ==
        gp.quicksum(x_ij[i, j] for i in factories if (i, j) in c_ij),
        name=f"FlowConservation_{j}"
    )
# customer demand
for k in customers:
    n.addConstr(
        gp.quicksum(y_ik[i, k] for i in factories if (i, k) in e_ik) +
        gp.quicksum(z_jk[j, k] for j in distribution_centers if (j, k) in f_jk) == d_k[k],
        name=f"CustomerDemand_{k}"
    )
n.update()

n.optimize()
if n.status == GRB.OPTIMAL:
    print("Optimal total cost:", n.objVal)
    for v in n.getVars():
        if v.X > 0:  # print only non-zero variables
            print(f"{v.VarName}: {v.X}")
else:
    print("No optimal solution found.")

Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16 rows, 44 columns and 96 nonzeros
Model fingerprint: 0xc86373ec
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 4e+05]
Presolve time: 0.01s
Presolved: 16 rows, 44 columns, 96 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+03   1.450000e+05   0.000000e+00      0s
       4    3.3500000e+04   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.02 seconds (0.00 work units)
Optimal objective  3.350000000e+04
Optimal total cost: 33500.0
x_ij[Cleveland,Tucson]: 40000.0
x_ij[Austin,Pittsburgh]: 70000.0
x_ij[Austin,Des Moines]: 

### 4. What is the effect of increasing the distribution center capacities on the overall cost? Carefully and fully explain your answer and how you arrived at it.
I created another updated model with the new DC capacities.

In [13]:
#updated dc capacity
new_t_j = {"Pittsburgh": 140000, "Boulder": 100000, "Des Moines": 200000, "Tucson": 80000}

#create model object
p = gp.Model('Shipping')

#specific obj function and time limit
p.ModelSense = GRB.MINIMIZE
p.setParam('TimeLimit', 600)
p.update()

#decision variables
x_ij = p.addVars(factories, distribution_centers, vtype=GRB.CONTINUOUS, name="x_ij")
y_ik = p.addVars(factories, customers, vtype=GRB.CONTINUOUS, name="y_ik")
z_jk = p.addVars(distribution_centers, customers, vtype=GRB.CONTINUOUS, name="z_jk")

#obj function
p.setObjective(
    gp.quicksum(c_ij[i, j] * x_ij[i, j] for i in factories for j in distribution_centers if c_ij[i, j] is not None) +
    gp.quicksum(e_ik[i, k] * y_ik[i, k] for i in factories for k in customers if e_ik[i, k] is not None) +
    gp.quicksum(f_jk[j, k] * z_jk[j, k] for j in distribution_centers for k in customers if f_jk[j, k] is not None)
)
p.update()
# factory capacity
for i in factories:
    p.addConstr(
        gp.quicksum(x_ij[i, j] for j in distribution_centers if (i, j) in c_ij) +
        gp.quicksum(y_ik[i, k] for k in customers if (i, k) in e_ik) <= s_i[i],
        name=f"FactoryCapacity_{i}"
    )
# dc capacity
for j in distribution_centers:
    p.addConstr(
        gp.quicksum(x_ij[i, j] for i in factories if (i, j) in c_ij) <= new_t_j[j],
        name=f"DCCapacity_{j}"
    )
# dc flow
for j in distribution_centers:
    p.addConstr(
        gp.quicksum(z_jk[j, k] for k in customers if (j, k) in f_jk) ==
        gp.quicksum(x_ij[i, j] for i in factories if (i, j) in c_ij),
        name=f"FlowConservation_{j}"
    )
# customer demand
for k in customers:
    p.addConstr(
        gp.quicksum(y_ik[i, k] for i in factories if (i, k) in e_ik) +
        gp.quicksum(z_jk[j, k] for j in distribution_centers if (j, k) in f_jk) == d_k[k],
        name=f"CustomerDemand_{k}"
    )
p.update()

p.optimize()
if p.status == GRB.OPTIMAL:
    print("Optimal total cost:", p.objVal)
    for v in n.getVars():
        if v.X > 0:  # print only non-zero variables
            print(f"{v.VarName}: {v.X}")
else:
    print("No optimal solution found.")

Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16 rows, 44 columns and 96 nonzeros
Model fingerprint: 0xe1b659b7
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve time: 0.01s
Presolved: 16 rows, 44 columns, 96 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+03   1.450000e+05   0.000000e+00      0s
       4    3.3500000e+04   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.350000000e+04
Optimal total cost: 33500.0
x_ij[Cleveland,Tucson]: 40000.0
x_ij[Austin,Pittsburgh]: 70000.0
x_ij[Austin,Des Moines]: 

### 5. What is the distribution pattern and total cost if the cost from Cleveland to Customer 1 changes to $1.25? Carefully and fully explain your answer and how you arrived at it.

I created a new model with this change by copying the original costs and then modifying the specific cost.

In [14]:
# updated transportation cost
updated_e_ik = e_ik.copy()
updated_e_ik[("Cleveland", "Customer1")] = 1.25  # change cost

q = gp.Model('Shipping_Updated_Cost')
q.ModelSense = GRB.MINIMIZE
q.setParam('TimeLimit', 600)
q.update()

# decision variables
x_ij = q.addVars(factories, distribution_centers, vtype=GRB.CONTINUOUS, name="x_ij")
y_ik = q.addVars(factories, customers, vtype=GRB.CONTINUOUS, name="y_ik")
z_jk = q.addVars(distribution_centers, customers, vtype=GRB.CONTINUOUS, name="z_jk")

# objective function
q.setObjective(
    gp.quicksum(c_ij[i, j] * x_ij[i, j] for i in factories for j in distribution_centers if c_ij[i, j] is not None) +
    gp.quicksum(updated_e_ik[i, k] * y_ik[i, k] for i in factories for k in customers if updated_e_ik[i, k] is not None) +
    gp.quicksum(f_jk[j, k] * z_jk[j, k] for j in distribution_centers for k in customers if f_jk[j, k] is not None)
)
q.update()

# constraints
for i in factories:
    q.addConstr(
        gp.quicksum(x_ij[i, j] for j in distribution_centers if (i, j) in c_ij) +
        gp.quicksum(y_ik[i, k] for k in customers if (i, k) in updated_e_ik) <= s_i[i],
        name=f"FactoryCapacity_{i}"
    )

for j in distribution_centers:
    q.addConstr(
        gp.quicksum(x_ij[i, j] for i in factories if (i, j) in c_ij) <= t_j[j],
        name=f"DCCapacity_{j}"
    )

for j in distribution_centers:
    q.addConstr(
        gp.quicksum(z_jk[j, k] for k in customers if (j, k) in f_jk) ==
        gp.quicksum(x_ij[i, j] for i in factories if (i, j) in c_ij),
        name=f"FlowConservation_{j}"
    )

for k in customers:
    q.addConstr(
        gp.quicksum(y_ik[i, k] for i in factories if (i, k) in updated_e_ik) +
        gp.quicksum(z_jk[j, k] for j in distribution_centers if (j, k) in f_jk) == d_k[k],
        name=f"CustomerDemand_{k}"
    )
q.update()

q.optimize()
if q.status == GRB.OPTIMAL:
    print("Optimal total cost:", q.objVal)
    for v in q.getVars():
        if v.X > 0:  # Only show non-zero decision variables
            print(f"{v.VarName}: {v.X}")
else:
    print("No optimal solution found.")


Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16 rows, 44 columns and 96 nonzeros
Model fingerprint: 0x6f26a049
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve time: 0.01s
Presolved: 16 rows, 44 columns, 96 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+03   1.450000e+05   0.000000e+00      0s
       4    3.3500000e+04   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.350000000e+04
Optimal total cost: 33500.0
x_ij[Cleveland,Tucson]: 40000.0
x_ij[Austin,Pittsburgh]: 70000.0
x_ij[Austin,Des Moines]: 

### 6. What is the distribution pattern and total cost if the cost from Cleveland to Customer 1 changes to $1.60? Carefully and fully explain your answer and how you arrived at it.

I made another model below, just to be safe.

In [29]:
# update transportation cost
updated_e_ik_r = e_ik.copy()
updated_e_ik_r[("Cleveland", "Customer1")] = 1.60  # Change cost for Cleveland to Customer 1

r = gp.Model('Shipping_Updated_Cost_r')
r.ModelSense = GRB.MINIMIZE
r.setParam('TimeLimit', 600)
r.update()

# decision variables
x_ij = r.addVars(factories, distribution_centers, vtype=GRB.CONTINUOUS, name="x_ij")
y_ik = r.addVars(factories, customers, vtype=GRB.CONTINUOUS, name="y_ik")
z_jk = r.addVars(distribution_centers, customers, vtype=GRB.CONTINUOUS, name="z_jk")

# objective function
r.setObjective(
    gp.quicksum(c_ij[i, j] * x_ij[i, j] for i in factories for j in distribution_centers if c_ij[i, j] is not None) +
    gp.quicksum(updated_e_ik_r[i, k] * y_ik[i, k] for i in factories for k in customers if updated_e_ik_r[i, k] is not None) +
    gp.quicksum(f_jk[j, k] * z_jk[j, k] for j in distribution_centers for k in customers if f_jk[j, k] is not None)
)
r.update()

# factory capacity
for i in factories:
    r.addConstr(
        gp.quicksum(x_ij[i, j] for j in distribution_centers if (i, j) in c_ij) +
        gp.quicksum(y_ik[i, k] for k in customers if (i, k) in updated_e_ik_r) <= s_i[i],
        name=f"FactoryCapacity_{i}"
    )

# distribution center capacity
for j in distribution_centers:
    r.addConstr(
        gp.quicksum(x_ij[i, j] for i in factories if (i, j) in c_ij) <= t_j[j],
        name=f"DCCapacity_{j}"
    )

# distribution center flow conservation
for j in distribution_centers:
    r.addConstr(
        gp.quicksum(z_jk[j, k] for k in customers if (j, k) in f_jk) ==
        gp.quicksum(x_ij[i, j] for i in factories if (i, j) in c_ij),
        name=f"FlowConservation_{j}"
    )

# customer demand
for k in customers:
    r.addConstr(
        gp.quicksum(y_ik[i, k] for i in factories if (i, k) in updated_e_ik_r) +
        gp.quicksum(z_jk[j, k] for j in distribution_centers if (j, k) in f_jk) == d_k[k],
        name=f"CustomerDemand_{k}"
    )
r.update()

# solve the model
r.optimize()

# display results
if r.status == GRB.OPTIMAL:
    print("Optimal total cost:", r.objVal)
    for v in r.getVars():
        if v.X > 0:  # Only show non-zero decision variables
            print(f"{v.VarName}: {v.X}")
else:
    print("No optimal solution found.")

Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16 rows, 44 columns and 96 nonzeros
Model fingerprint: 0x007cb1a1
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve time: 0.00s
Presolved: 16 rows, 44 columns, 96 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+03   1.450000e+05   0.000000e+00      0s
       4    3.3500000e+04   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.350000000e+04
Optimal total cost: 33500.0
x_ij[Cleveland,Tucson]: 40000.0
x_ij[Austin,Pittsburgh]: 70000.0
x_ij[Austin,Des Moines]: 

### 7. Is it possible to meet all customer preferences regarding suppliers? Carefully and fully explain your answer and how you arrived at it.

In [35]:
# preferred supplier table
preferred_suppliers = {
    "Customer1": ["Cleveland"],
    "Customer2": ["Pittsburgh"],
    "Customer3": None,
    "Customer4": None,
    "Customer5": ["Boulder"],
    "Customer6": ["Des Moines", "Tucson"],
}

# model
model = gp.Model("CustomerPreferences")
model.ModelSense = GRB.MINIMIZE
model.setParam("TimeLimit", 600)

# decision variables
x_ij = model.addVars(factories, distribution_centers, vtype=GRB.CONTINUOUS, name="x_ij")
y_ik = model.addVars(factories, customers, vtype=GRB.CONTINUOUS, name="y_ik")
z_jk = model.addVars(distribution_centers, customers, vtype=GRB.CONTINUOUS, name="z_jk")

# obj function
model.setObjective(
    gp.quicksum(c_ij[i, j] * x_ij[i, j] for i in factories for j in distribution_centers if c_ij[i, j] is not None) +
    gp.quicksum(e_ik[i, k] * y_ik[i, k] for i in factories for k in customers if e_ik[i, k] is not None) +
    gp.quicksum(f_jk[j, k] * z_jk[j, k] for j in distribution_centers for k in customers if f_jk[j, k] is not None)
)

# factory capacity
for i in factories:
    model.addConstr(
        gp.quicksum(x_ij[i, j] for j in distribution_centers if (i, j) in c_ij) +
        gp.quicksum(y_ik[i, k] for k in customers if (i, k) in e_ik) <= s_i[i],
        name=f"FactoryCapacity_{i}"
    )

# dc capacity
for j in distribution_centers:
    model.addConstr(
        gp.quicksum(x_ij[i, j] for i in factories if (i, j) in c_ij) <= t_j[j],
        name=f"DCCapacity_{j}"
    )

# flow conservation
for j in distribution_centers:
    model.addConstr(
        gp.quicksum(z_jk[j, k] for k in customers if (j, k) in f_jk) ==
        gp.quicksum(x_ij[i, j] for i in factories if (i, j) in c_ij),
        name=f"FlowConservation_{j}"
    )

# customer demand
for k in customers:
    model.addConstr(
        gp.quicksum(y_ik[i, k] for i in factories if (i, k) in e_ik) +
        gp.quicksum(z_jk[j, k] for j in distribution_centers if (j, k) in f_jk) == d_k[k],
        name=f"CustomerDemand_{k}"
    )

# NEW: PREFERENCES
for k, prefs in preferred_suppliers.items():
    if prefs:
        allowed_sources = [
            y_ik[i, k] for i in factories if i in prefs if (i, k) in e_ik
        ] + [
            z_jk[j, k] for j in distribution_centers if j in prefs if (j, k) in f_jk
        ]
        model.addConstr(
            gp.quicksum(allowed_sources) == d_k[k],
            name=f"CustomerPreference_{k}"
        )

model.update()
model.optimize()
if model.status == GRB.OPTIMAL:
    print(f"All customer preferences can be satisfied. Optimal cost: {model.objVal}")
    for v in model.getVars():
        if v.X > 0:
            print(f"{v.VarName}: {v.X}")
else:
    print("It is not possible to meet all customer preferences.")

Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 20 rows, 44 columns and 101 nonzeros
Model fingerprint: 0x12f2d69d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve removed 9 rows and 24 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Infeasible model
It is not possible to meet all customer preferences.


### 8. If it is impossible to meet all customer preferences, what is the best distribution pattern to satisfy as many of the preferences as possible? What is the extra cost of doing this?

In [37]:
# preferred supplier table
preferred_suppliers = {
    "Customer1": ["Cleveland"],
    "Customer2": ["Pittsburgh"],
    "Customer3": None,
    "Customer4": None,
    "Customer5": None, # REMOVED TO TEST
    "Customer6": ["Des Moines", "Tucson"],
}

# model
model = gp.Model("CustomerPreferences")
model.ModelSense = GRB.MINIMIZE
model.setParam("TimeLimit", 600)

# decision variables
x_ij = model.addVars(factories, distribution_centers, vtype=GRB.CONTINUOUS, name="x_ij")
y_ik = model.addVars(factories, customers, vtype=GRB.CONTINUOUS, name="y_ik")
z_jk = model.addVars(distribution_centers, customers, vtype=GRB.CONTINUOUS, name="z_jk")

# obj function
model.setObjective(
    gp.quicksum(c_ij[i, j] * x_ij[i, j] for i in factories for j in distribution_centers if c_ij[i, j] is not None) +
    gp.quicksum(e_ik[i, k] * y_ik[i, k] for i in factories for k in customers if e_ik[i, k] is not None) +
    gp.quicksum(f_jk[j, k] * z_jk[j, k] for j in distribution_centers for k in customers if f_jk[j, k] is not None)
)

# factory capacity
for i in factories:
    model.addConstr(
        gp.quicksum(x_ij[i, j] for j in distribution_centers if (i, j) in c_ij) +
        gp.quicksum(y_ik[i, k] for k in customers if (i, k) in e_ik) <= s_i[i],
        name=f"FactoryCapacity_{i}"
    )

# dc capacity
for j in distribution_centers:
    model.addConstr(
        gp.quicksum(x_ij[i, j] for i in factories if (i, j) in c_ij) <= t_j[j],
        name=f"DCCapacity_{j}"
    )

# flow conservation
for j in distribution_centers:
    model.addConstr(
        gp.quicksum(z_jk[j, k] for k in customers if (j, k) in f_jk) ==
        gp.quicksum(x_ij[i, j] for i in factories if (i, j) in c_ij),
        name=f"FlowConservation_{j}"
    )

# customer demand
for k in customers:
    model.addConstr(
        gp.quicksum(y_ik[i, k] for i in factories if (i, k) in e_ik) +
        gp.quicksum(z_jk[j, k] for j in distribution_centers if (j, k) in f_jk) == d_k[k],
        name=f"CustomerDemand_{k}"
    )

# NEW: PREFERENCES
for k, prefs in preferred_suppliers.items():
    if prefs:
        allowed_sources = [
            y_ik[i, k] for i in factories if i in prefs if (i, k) in e_ik
        ] + [
            z_jk[j, k] for j in distribution_centers if j in prefs if (j, k) in f_jk
        ]
        model.addConstr(
            gp.quicksum(allowed_sources) == d_k[k],
            name=f"CustomerPreference_{k}"
        )

model.update()
model.optimize()
if model.status == GRB.OPTIMAL:
    print(f"All customer preferences can be satisfied. Optimal cost: {model.objVal}")
    for v in model.getVars():
        if v.X > 0:
            print(f"{v.VarName}: {v.X}")
else:
    print("It is not possible to meet all customer preferences.")

Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 19 rows, 44 columns and 100 nonzeros
Model fingerprint: 0xcf8d7bba
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve removed 6 rows and 17 columns
Presolve time: 0.01s
Presolved: 13 rows, 27 columns, 62 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0300000e+05   1.436750e+04   0.000000e+00      0s
       5    1.3450000e+05   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.345000000e+05
All customer preferences can be satisfied. Optimal cost: 134500.0
x_ij[Clevela